# Effortless Accuracy: Unlocking the Power of Baseline Forecasts

### Required Packages

In [ ]:
import os

import pandas as pd
from statsforecast import StatsForecast
from statsforecast.models import (
    HistoricAverage,
    Naive,
    SeasonalNaive,
    WindowAverage,
)
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import rmse

os.environ["NIXTLA_ID_AS_COL"] = "1"

### Getting the data
Tourism data (from the R-tsibble package), but with only 3 regions included.

In [ ]:
df = pd.read_csv("../data/tourism.csv")
df["ds"] = pd.PeriodIndex(df["ds"], freq="Q").to_timestamp()
df

### Splitting data into test and train

In [ ]:
test_df = df.groupby("unique_id", group_keys=False).tail(4)
train_df = df[~df.index.isin(test_df.index)]

### Setting up the baseline models and training them on the train data.

In [ ]:
# Define forecast horizon
h = 4

models = [
    HistoricAverage(),
    Naive(),
    SeasonalNaive(season_length = 4), # Quarterly data seasonality = 4
    WindowAverage(window_size=4)
]

sf = StatsForecast(
    models=models,
    freq="QS",
)

sf.fit(train_df)

### Predicting for each of the model

In [ ]:
pred_df = sf.predict(h=h)

In [ ]:
sf.plot(df, pred_df)

### Evaluate the models

In [ ]:
accuracy_df =  pd.merge(test_df, pred_df, how = "left", on = ["unique_id", "ds"])
evaluate(accuracy_df, metrics=[rmse])